Copyright **`(c)`** 2024 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free under certain conditions — see the [`license`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [1058]:
import logging
from itertools import combinations
import pandas as pd
import numpy as np
from geopy.distance import geodesic
import networkx as nx
import random
from tqdm.auto import tqdm
from matplotlib import pyplot as plt
from itertools import accumulate
from icecream import ic

logging.basicConfig(level=logging.DEBUG)

In [ ]:
CITIES = pd.read_csv('cities/italy.csv', header=None, names=['name', 'lat', 'lon'])
DIST_MATRIX = np.zeros((len(CITIES), len(CITIES)))
for c1, c2 in combinations(CITIES.itertuples(), 2):
    DIST_MATRIX[c1.Index, c2.Index] = DIST_MATRIX[c2.Index, c1.Index] = geodesic(
        (c1.lat, c1.lon), (c2.lat, c2.lon)
    ).km
CITIES.head()

## Lab2 - TSP

https://www.wolframcloud.com/obj/giovanni.squillero/Published/Lab2-tsp.nb

In [1060]:
def tsp_cost(tsp):
    assert tsp[0] == tsp[-1]
    assert set(tsp) == set(range(len(CITIES)))

    tot_cost = 0
    for c1, c2 in zip(tsp, tsp[1:]):
        tot_cost += DIST_MATRIX[c1, c2]
    return tot_cost

def fitness(solution: np.ndarray):
    return -tsp_cost(solution)

## First Greedy Algorithm

In [1061]:
def greedy(city):
    visited = np.full(len(CITIES), False)
    dist = DIST_MATRIX.copy()
    visited[city] = True
    tsp = list()
    tsp.append(int(city))
    while not np.all(visited):
        dist[:, city] = np.inf
        closest = np.argmin(dist[city])
        logging.debug(
            f"step: {CITIES.at[city,'name']} -> {CITIES.at[closest,'name']} ({DIST_MATRIX[city,closest]:.2f}km)"
        )
        visited[closest] = True
        city = closest
        tsp.append(int(city))
    logging.debug(
        f"step: {CITIES.at[tsp[-1],'name']} -> {CITIES.at[tsp[0],'name']} ({DIST_MATRIX[tsp[-1],tsp[0]]:.2f}km)"
    )
    tsp.append(tsp[0])


    logging.info(f"result: Found a path of {len(tsp)-1} steps, total length {tsp_cost(tsp):.2f}km")
    return tsp

## Simulated Annealing

In [ ]:
#Ho provato a inizializzare la popolazione con un simulated annealing semplice ma i risultati non sono stati soddisfacenti.

'''
MAX_STEPS = 1000
NUM_INDIVIDUALS = 10

#EA
population = []

#Simulated Annealing

for s in range(NUM_INDIVIDUALS):

    #Soluzione casuale
    visited = np.full(len(CITIES), False)
    dist = DIST_MATRIX.copy()
    solution = np.arange(len(CITIES))
    np.random.shuffle(solution)
    solution = np.append(solution, solution[0])
    #ic("Soluzione random",solution)

    for n in tqdm(range(MAX_STEPS)):
        # TWEAK!
        new_solution = solution.copy()
        #print(new_solution)
        while True:
            index1 = np.random.randint(0, len(CITIES))
            index2 = np.random.randint(0, len(CITIES))
            if index1 != index2:
                break

        new_solution[index1], new_solution[index2] = new_solution[index2], new_solution[index1]
        if index1 == 0:
            new_solution[-1] = new_solution[0]
        if index2 == 0:
            new_solution[-1] = new_solution[0]
        if index1 == len(CITIES):
            new_solution[0] = new_solution[-1]
        if index2 == len(CITIES):
            new_solution[0] = new_solution[-1]

        #print(new_solution)

        history.append(fitness(new_solution))
        if tsp_cost(new_solution) < tsp_cost(solution):
            solution = new_solution

    population.append(solution)
    '''
'''
# That's all...
ic(fitness(solution))
ic(history.index(fitness(solution)))
ic(solution)
'''

'''plt.figure(figsize=(14, 8))
plt.plot(
    range(len(history)),
    list(accumulate(history, max)),
    color="red",
)
_ = plt.scatter(range(len(history)), history, marker=".")'''



__Generazione popolazione tramite Greedy__

In [ ]:
population = []

for i in range(CITIES.shape[0]):
    population.append(np.array(greedy(i)))

In [1064]:
max_segment_length = (CITIES.shape[0] // 2)
dist = DIST_MATRIX.copy()

def calculate_cost_ea(tour):
    """ Calcola il costo totale del percorso """
    return sum(dist[tour[i], tour[i + 1]] for i in range(len(tour) - 1)) + dist[tour[-1], tour[0]]

def fitness_ea(tour):
    """ Calcola il fitness di un percorso (minimizzando il costo) """
    return -calculate_cost_ea(tour)

def trim_last_city(population):
    """ Rimuove l'ultima città (che è anche quella di partenza) da ogni percorso """
    return [tour[:-1] for tour in population]

def inver_over_operator(parent1, parent2):

    segment_length = random.randint(2, max_segment_length)
    index1 = random.randint(0, len(parent1) - segment_length)
    index2 = index1 + segment_length-1
    segment = parent1[index1+1:index2][::-1]

    child = np.insert(segment, 0, [parent1[index1], parent1[index2]])
    diff = [x for x in parent2 if x not in child]
    child = np.concatenate([diff, child])
    
    return child

def rank_population(population):
    """ Ordina la popolazione in base al fitness """
    sorted_population = sorted(population, key=lambda x: calculate_cost_ea(x))
    return sorted_population


def choose_parents(population):
    """ Sceglie due genitori dalla popolazione """

    # Definisci un array di probabilità decrescente
    probabilities = np.linspace(1, 0.1, len(population))  # Probabilità decrescenti
    #x = np.arange(len(population))
    #probabilities = np.exp(-x)
    probabilities /= probabilities.sum()  # Normalizza le probabilità in modo che sommino a 1

    # Seleziona un elemento con probabilità decrescente
    selected_parents = np.random.choice(np.arange(len(population)), p=probabilities, size = 2)
    #print("Elemento selezionato:", selected_parents)


    return selected_parents


In [ ]:
MAX_STEPS_XOVER = range(1000)
MAX_STEPS_EA = range(100)
NUM_INDIVIDUALS = CITIES.shape[0]

population = trim_last_city(population)
offsprings = []

population = rank_population(population)
for _ in MAX_STEPS_EA:
    for _ in MAX_STEPS_XOVER:

        selected_parents = choose_parents(population)

        offsprings.append(inver_over_operator(population[selected_parents[0]], population[selected_parents[1]]))

    population = np.array(population)

    population = np.concatenate([population, offsprings])
    population = rank_population(population)
    population = population[:NUM_INDIVIDUALS]

ic(calculate_cost_ea(population[0]))